## Hand Made Neural Network

https://144.34.140.210:8579/notebooks/Git_Qiliang/ML_ANN_Gradiant_Decent/Backpropagation.ipynb

In [ ]:
import pdb
bp = pdb.set_trace

In [ ]:
import numpy as np

In [ ]:
import random
import uuid
import numpy as np



In [11]:
a = [[1,2,3,4,5,6,7,8,9,10],]
b = [12,3,4,5,1,8,5,12,98,2]
a = np.array(a)
b = np.array(b)

a*b

array([[ 12,   6,  12,  20,   5,  48,  35,  96, 882,  20]])

In [12]:
a*b.T

array([[ 12,   6,  12,  20,   5,  48,  35,  96, 882,  20]])

In [14]:
a.T*b

array([[ 12,   3,   4,   5,   1,   8,   5,  12,  98,   2],
       [ 24,   6,   8,  10,   2,  16,  10,  24, 196,   4],
       [ 36,   9,  12,  15,   3,  24,  15,  36, 294,   6],
       [ 48,  12,  16,  20,   4,  32,  20,  48, 392,   8],
       [ 60,  15,  20,  25,   5,  40,  25,  60, 490,  10],
       [ 72,  18,  24,  30,   6,  48,  30,  72, 588,  12],
       [ 84,  21,  28,  35,   7,  56,  35,  84, 686,  14],
       [ 96,  24,  32,  40,   8,  64,  40,  96, 784,  16],
       [108,  27,  36,  45,   9,  72,  45, 108, 882,  18],
       [120,  30,  40,  50,  10,  80,  50, 120, 980,  20]])

In [ ]:
# x = np.array(random.choices(range(100), k=10))
# x = x/x.sum()
# vector = [2, 24, 31, 81, 2, 76, 57, 81, 79, 76]
# vector = np.array(vector)
# vector = vector/vector.sum()
# # y = x.dot(vector) + random.randrange(-10,10)
# y = x.dot(vector) + (random.random()-0.5)*0.2

# print(y)

In [381]:
def relu(x):
    if x>0:
        return x
    else:
        return 0
activation_func = np.vectorize(relu)  
         
class Weight:
    def __init__(self,m,n):
        self.m = m
        self.n = n
        self.val = np.random.randn(m,n)
        self.normalize()
        
    def normalize(self):
        self.val = self.val/self.val.sum()

class Layer:
    def __init__(self,n,id=None):
        self.val = None
        self.id = id
        self.input_weight = None
        self.output_weight = None
        self.pre_layer = None
        self.next_layer = None
        self.n = n
        self.delta_weight = None
        
        
    def full_connect(self,other):
        m = self.n
        n = other.n
        weight = Weight(m,n)
        self.output_weight = weight
        other.input_weight = weight
        
        self.next_layer = other
        other.pre_layer = self
    
    def stats(self):
        print("n:",self.n)
        if self.output_weight:
            print('Weight:',self.output_weight.val.shape)
        print()
            
    def pull_in(self,input_list):
        self.val = np.array([input_list,])
        self.val = self.val/self.val.sum()
            
    def forwarding(self):
#         print(self.id,'forwarding')
        val = self.val.dot(self.output_weight.val)
        try:
            if self.next_layer:
                self.next_layer.val = activation_func(val)      
        except:
#             bp()
            raise
        
    def backwarding(self, error_term, learning_rate):
#         print(self.id,'backwarding')
        pre = self.pre_layer
        K1 = pre.output_weight.val
#         print(K1.shape)
        while(pre.pre_layer):
            pre = pre.pre_layer
            K1 = pre.output_weight.val.dot(K1)
            
        K1 = pre.val.dot(K1)
#         print('K1:',K1.shape)


        next_layer = self.next_layer
        if next_layer.output_weight:
            K2 = next_layer.output_weight.val
            while(next_layer.next_layer.output_weight):
                next_layer = next_layer.next_layer
                K2 = K2.dot(next_layer.output_weight.val) 
#             print("K2:",K2.shape)
        else:
            K2 = np.array([[1]])
            
            
#         return K1,K2
#         bp()
        derivative = K1.T.dot(K2.T)
#         print("Derivative:",derivative.shape)
#         return derivative
    
        delta_weight = learning_rate*(-1)*(error_term)*derivative
#         bp()
        if type(self.delta_weight)!=type(None):
            self.delta_weight += delta_weight
        else:
            self.delta_weight = delta_weight
    
    def update_weights(self):
        if type(self.delta_weight)!=type(None):
            self.output_weight.val += self.delta_weight
            self.delta_weight = None
    
class Network:
    def __init__(self,input_layer,output_layer):
        self.input_layer = input_layer
        self.output_layer = output_layer
              
    def batch_forwarding(self):
        cur = self.input_layer
        while(cur.next_layer):
            cur.forwarding()
            cur = cur.next_layer
             
    def get_error_term(self,ground_truth_list):      
        error = self.output_layer.val-ground_truth_list
        return error
    
    def batch_backwarding(self, error_term, learning_rate=0.01):
        cur = self.input_layer.next_layer
        while(cur and cur.output_weight):
            cur.backwarding(error_term, learning_rate)
            cur = cur.next_layer
            
    def one_train(self,x,y):
        input_layer.pull_in(x)
        self.batch_forwarding()
        error_term = self.get_error_term(y)
        self.batch_backwarding(error_term,0.001)
        
        
    def batch_update_weights(self):
        cur = self.input_layer.next_layer
        while(cur and cur.output_weight):
            cur.update_weights() 
            cur = cur.next_layer

    def batch_train(self,X,Y,batch_size=20):
        i=0
        for i,(x,y) in enumerate(zip(X,Y)):
            self.one_train(x,y)
            if i%batch_size==0:
                self.batch_update_weights()
        self.batch_update_weights()
        
        
    def predict(self,x):
        input_layer.pull_in(x)
        self.batch_forwarding()
        return self.output_layer.val
        
# # Networks
# input_layer = Layer(10,'in')
# l1 = Layer(10,'l1')
# l2 = Layer(50,'l2')
# l3 = Layer(50,'l3')
# l4 = Layer(10,'l4')
# output_layer = Layer(1,'out')


# input_layer.full_connect(l1)
# l1.full_connect(l2)
# l2.full_connect(l3)
# l3.full_connect(l4)
# l4.full_connect(output_layer)

# x = random.choices(range(100), k=10)

# print(x)
# y = np.array([[1]])

# nk = Network(input_layer,output_layer)
# nk.one_train(x,y)



In [399]:
# Networks
input_layer = Layer(10,'in')
l1 = Layer(10,'l1')
l2 = Layer(50,'l2')
l3 = Layer(50,'l3')
l4 = Layer(10,'l4')
output_layer = Layer(1,'out')


input_layer.full_connect(l1)
l1.full_connect(l2)
l2.full_connect(l3)
l3.full_connect(l4)
l4.full_connect(output_layer)


nk = Network(input_layer,output_layer)

In [815]:
vector = [2, 24, 31, 81, 2, 76, 57, 81, 79, 76]
vector = np.array(vector)
vector = vector/vector.sum()

X = []
Y = []
for i in range(1000):
    x = np.array(random.choices(range(100), k=10))
    x = x/x.sum()
    y = x.dot(vector) + (random.random()-0.5)*0.05
    X.append(x)
    Y.append(y)

In [816]:
nk.batch_train(X,Y)

In [817]:
x = np.array(random.choices(range(100), k=10))
x = x/x.sum()
y_percise = x.dot(vector) 
y_real = x.dot(vector) + (random.random()-0.5)*0.05
y_predict = nk.predict(x)

print('x:',x)
print('y_percise:',y_percise)
print('y_real:',y_real)
print('y_predict:',y_predict)

x: [0.14411248 0.10544815 0.14235501 0.00527241 0.14235501 0.04393673
 0.11775044 0.09314587 0.10193322 0.10369069]
y_percise: 0.08147889828430949
y_real: 0.0935566139865746
y_predict: [[0.10146774]]


In [751]:
# nk.batch_train(X,Y)

In [93]:
l4.output_weight.val

array([[-0.19269106],
       [ 1.39741747],
       [ 1.04548973],
       [-0.20258485],
       [-1.95243922],
       [-0.34084244],
       [-0.39326694],
       [-1.07620167],
       [ 0.01801484],
       [ 1.91752749]])

In [74]:
l3.output_weight.val

array([[ 7.99489773e-02, -1.28883481e-01, -1.08758202e-01,
        -2.83299369e-01,  1.30403852e-01,  4.64624820e-02,
         8.49860513e-02, -8.26779228e-02,  4.03718589e-02,
        -4.94254108e-02],
       [ 1.28535677e-01,  1.92719495e-01, -1.67727969e-01,
         3.02950506e-01,  1.82792013e-01, -2.54098717e-02,
         8.58592590e-02,  1.74493863e-01, -1.65312083e-01,
        -1.15622542e-01],
       [-3.22899721e-01,  6.27529289e-04, -2.81818225e-02,
         2.34980358e-01, -9.53289463e-02, -1.41846785e-02,
        -3.38269723e-01, -2.10634137e-01,  1.13859656e-01,
        -4.16460248e-04],
       [ 6.62219881e-01,  6.42108937e-02,  8.98429328e-02,
        -1.31240825e-01, -4.08319459e-02,  4.94124012e-02,
         1.15341234e-01, -2.43517453e-01, -1.28914020e-01,
         3.01001457e-01],
       [ 6.84340166e-02,  8.86964974e-03, -2.96260648e-01,
        -1.24121282e-02, -1.78535662e-01, -9.22442151e-02,
         2.69662228e-01, -8.34643276e-03, -2.21792079e-01,
        -1.

In [75]:
l2.output_weight.val

array([[ 0.08510723, -0.01121689, -0.01802326, ..., -0.03507009,
         0.05629599,  0.00848492],
       [-0.02020317,  0.03367152,  0.00655599, ...,  0.0130693 ,
         0.03150085,  0.04805167],
       [ 0.00438371,  0.01148298,  0.00219288, ...,  0.00607755,
         0.05995458,  0.03074161],
       ...,
       [-0.00567585,  0.04768555, -0.00900835, ..., -0.0961654 ,
         0.04391923, -0.00955867],
       [-0.01522867,  0.01316334,  0.02044633, ..., -0.0189906 ,
         0.03178518,  0.02229477],
       [ 0.03394011,  0.02953836, -0.05973158, ..., -0.02565085,
        -0.06868927,  0.07607553]])

In [76]:
l1.output_weight.val

array([[-0.01605551, -0.10052804,  0.00152309,  0.18569246,  0.12513207,
         0.13026984,  0.09429307, -0.12387047,  0.13553875, -0.04172468,
         0.00646714, -0.13930396,  0.10786934,  0.14239545, -0.10888101,
         0.15701333,  0.08263467,  0.09804768, -0.34027071,  0.14594672,
         0.18086073,  0.0558452 ,  0.06015864, -0.10535668,  0.10818266,
         0.27277555,  0.05553154, -0.02210709, -0.07353723,  0.03661818,
        -0.10922774, -0.03277336,  0.07837544, -0.17448074, -0.02590112,
        -0.10449554, -0.18055676, -0.09745217,  0.1167623 ,  0.11761512,
         0.06978527, -0.13538102, -0.03521796, -0.06115944, -0.01217703,
        -0.08144725,  0.23943186, -0.25066258, -0.12022857,  0.08354386],
       [-0.0180743 , -0.23592704, -0.13231129, -0.03091949,  0.00159445,
         0.02342056,  0.04706227, -0.16184306,  0.03507666,  0.22523181,
         0.18300497, -0.06247585,  0.01097271,  0.08076567,  0.05436707,
        -0.24852148, -0.00296589, -0.08440308, -0.

In [31]:
l4.val

array([[0.00058268, 0.        , 0.004316  , 0.00616437, 0.00502247,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [330]:
l3.val

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [ ]:
"""
如何设置可以使信号不至于消失？？？？

同时避免参数爆炸

可行的路径、重复感受和验证的，嫁接的方法

"""

In [223]:
nk.predict(x)

array([[0]])

In [214]:
l1.stats()
l2.stats()
l3.stats()
l4.stats()

n: 10
Weight: (10, 50)

n: 50
Weight: (50, 50)

n: 50
Weight: (50, 10)

n: 10
Weight: (10, 1)



In [215]:
print('input_layer:',[round(x,2) for x in input_layer.val[0]])
print('l1:',[round(x,2) for x in l1.val[0]])
print('l2:',[round(x,2) for x in l2.val[0]])
print('l3:',[round(x,2) for x in l3.val[0]])
print('l4:',[round(x,2) for x in l4.val[0]])
print('output_layer:',[round(x,2) for x in output_layer.val[0]])

input_layer: [85, 78, 24, 50, 51, 95, 1, 22, 51, 65]
l1: [24.03, 7.7, 37.9, 0.0, 45.19, 0.0, 82.94, 0.0, 0.0, 0.0]
l2: [2.99, 0.0, 3.95, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7, 0.0, 0.0, 6.15, 0.0, 12.57, 25.25, 15.12, 0.0, 0.0, 0.0, 11.79, 19.67, 28.17, 0.0, 5.11, 6.01, 0.0, 0.0, 2.56, 13.08, 35.02, 0.0, 3.71, 62.84, 28.64, 51.16, 0.54, 0.0, 33.64, 0.0, 19.34, 0.0, 0.0, 23.39, 15.72, 0.0, 0.52, 15.9, 0.0, 0.0, 20.21]
l3: [0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 0, 0, 0, 2]
l4: [0.57, 0.0, 0.0, 0.38, 0.0, 0.0, 0.0, 0.0, 0.3, 0.08]
output_layer: [0.15]


In [216]:
l1.delta_weight

array([[ 3.03586222e-04,  1.34174545e-04,  1.22835635e-03,
         7.51947139e-04,  2.10621331e-04, -2.25281513e-04,
         4.58187184e-04, -2.46946156e-04,  5.46989560e-04,
        -3.43687642e-04, -1.59704308e-03,  1.14776961e-04,
        -2.20332758e-04,  1.64067880e-03, -6.79567001e-05,
        -5.21673877e-04,  8.03791896e-04,  1.42830370e-03,
        -1.02715743e-04,  1.80930581e-04, -1.24633486e-03,
        -8.01996503e-04, -1.80922030e-04, -3.42926879e-04,
         1.10753069e-04,  7.04940749e-04,  9.13847454e-05,
        -9.89001596e-04,  3.15081050e-04,  7.34962703e-04,
        -1.31275881e-03,  1.44556152e-03, -1.49704793e-04,
         1.29837762e-03,  1.40979862e-03, -5.61983500e-04,
         9.43563031e-04, -6.12721008e-05,  6.11976414e-04,
        -7.71239116e-04,  7.29477931e-05,  9.56180302e-04,
        -4.16107126e-04,  9.52857139e-04,  5.78770502e-04,
         7.90586184e-04,  1.39235811e-03,  6.02323722e-04,
        -3.59216151e-04, -1.20803097e-04],
       [ 9.72

In [217]:
l2.delta_weight

array([[ 1.81545014e-03, -5.00561458e-03,  6.79693171e-03, ...,
        -5.55600742e-03, -4.51932771e-04,  1.76388167e-03],
       [-6.45120604e-03,  1.77874624e-02, -2.41529118e-02, ...,
         1.97432846e-02,  1.60594409e-03, -6.26795740e-03],
       [ 3.19865500e-03, -8.81942930e-03,  1.19755642e-02, ...,
        -9.78917051e-03, -7.96263687e-04,  3.10779615e-03],
       ...,
       [ 1.17568665e-04, -3.24163916e-04,  4.40169727e-04, ...,
        -3.59807391e-04, -2.92671948e-05,  1.14229089e-04],
       [-5.76694087e-03,  1.59007856e-02, -2.15910658e-02, ...,
         1.76491580e-02,  1.43560515e-03, -5.60312901e-03],
       [ 7.58638396e-03, -2.09174096e-02,  2.84029469e-02, ...,
        -2.32173855e-02, -1.88853192e-03,  7.37089022e-03]])

In [159]:
l1.stats()

n: 10


In [160]:
# l2.output_edges
l2.stats()

n: 50


In [37]:
l3.stats()

Num nodes: 50
Input Edges: 2500
Output Edges: 50


In [38]:
l4.stats()

Num nodes: 1
Input Edges: 50
Output Edges: 0
